In [1]:
from NoisyCircuits import QuantumCircuit as QC
from NoisyCircuits.utils.GetNoiseModel import GetNoiseModel
import pickle
import os
import json
import timeit

2026-01-07 12:31:24,353	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
token = json.load(open(os.path.join(os.path.expanduser("~"), "ibm_api.json"), "r"))["apikey"] # Replace with your IBM Quantum token
backend_name = "ibm_fez"
num_qubits = 11
num_cores = 1
num_trajectories = 20
threshold = 1e-2
jsonize = True
verbose = True
# Options: "heron", "eagle". Note that "eagle" is now deprecated and only available in simulation mode (i.e., no noise model from hardware).
qpu_type = "heron" 

In [3]:
if qpu_type == "eagle":
    noise_model = pickle.load(open("../noise_models/Noise_Model_Eagle_QPU.pkl", "rb"))
elif qpu_type == "heron":
    noise_model = pickle.load(open("../noise_models/Noise_Model_Heron_QPU.pkl", "rb"))
else:
    raise ValueError("Invalid qpu_type. Choose either 'heron' or 'eagle'.")

In [4]:
nqc = QC(num_qubits=num_qubits, 
         noise_model=noise_model, 
         num_cores=num_cores,
         backend_qpu_type="heron", 
         num_trajectories=num_trajectories, 
         threshold=threshold, 
         jsonize=jsonize,
         verbose=verbose)

Available qubits in roerror_map: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155]
Requested qubits: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
Completed Extraction of Measurement Errors.
Completed Extraction of two-qubit gate Errors.
Starting post-processing on Single Qubit Errors.
Completed post-processing on Single Qubit Errors.
Processing two-qubit gate errors.
Qubit pair (0, 1): 

In [10]:
mat = nqc.single_qubit_instructions[0]["x"]["kraus_operators"][0]

In [6]:
import numpy as np

In [8]:
vec = np.random.normal(size=(2**num_qubits,)) + 1j * np.random.normal(size=(2**num_qubits,))
vec /= np.linalg.norm(vec)

In [30]:
vals = mat.data
indices = mat.indices
indptr = mat.indptr
mat_custom = (vals, indices, indptr)

In [64]:
import numba

In [92]:
def matrix_vector_scipy(mat, vec):
    res = mat.dot(vec)
    return res

def matrix_vector_custom(mat, vec):
    res = np.add.reduceat(mat[0] * vec[mat[1]], mat[2][:-1])
    return res

@numba.njit
def matrix_vector_custom_numba(mat, vec):
    vals, indices, indptr = mat
    res = np.zeros_like(vec)
    for i in range(res.shape[0]):
        for j in range(indptr[i], indptr[i + 1]):
            res[i] += vals[j] * vec[indices[j]]
    return res

In [93]:
times_scipy = timeit.repeat(lambda:matrix_vector_scipy(mat,vec), repeat=100000, number=1)

In [94]:
times_custom = timeit.repeat(lambda:matrix_vector_custom(mat_custom, vec), repeat=100000, number=1)

In [95]:
times_numba = timeit.repeat(lambda:matrix_vector_custom_numba(mat_custom, vec), repeat=100000, number=1)

In [96]:
print("SciPy sparse matrix-vector multiplication times:\nMean: {:.6f} s, Std: {:.6f} s\n".format(np.mean(times_scipy), np.std(times_scipy)))
print("Custom sparse matrix-vector multiplication times:\nMean: {:.6f} s, Std: {:.6f} s\n".format(np.mean(times_custom), np.std(times_custom)))
print("Custom sparse matrix-vector multiplication times with Numba:\nMean: {:.6f} s, Std: {:.6f} s".format(np.mean(times_numba), np.std(times_numba)))

SciPy sparse matrix-vector multiplication times:
Mean: 0.000012 s, Std: 0.000006 s

Custom sparse matrix-vector multiplication times:
Mean: 0.000023 s, Std: 0.000004 s

Custom sparse matrix-vector multiplication times with Numba:
Mean: 0.000012 s, Std: 0.000664 s


In [98]:
np.allclose(matrix_vector_scipy(mat,vec), matrix_vector_custom(mat_custom, vec))

True

In [97]:
np.allclose(matrix_vector_scipy(mat, vec), matrix_vector_custom_numba(mat_custom, vec))

True

In [91]:
nqc.shutdown()